In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
import sys
from Bio import SeqIO
from data_generator import data_generator

#from google.colab import drive

In [ ]:
class LSTM_model(nn.Module):
    def __init__(self, input_size, hidden_layer_size, nr_hidden_layers):
        super(LSTM_model, self).__init__()
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.nr_hidden_layers = nr_hidden_layers
        
        self.model = nn.LSTM(input_size, hidden_layer_size, nr_hidden_layers)
        
        self.linear = nn.Linear(hidden_layer_size, input_size)
        
    def forward(self, input_data):
        lstm_out, _ = self.model(input_data)
        tag_space = self.linear(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
acids = "ACDEFGHIKLMNPQRSTUVWY-"

hidden_dim = 500
hidden_layers = 1

model = LSTM_model(len(acids), hidden_dim, hidden_layers)#.cuda()
loss_function = nn.CrossEntropyLoss()
#loss_function = nn.NLLLoss()
optimiser = optim.SGD(model.parameters(), lr=0.1)

large_file = "uniref50.fasta"
small_file = "100k_rows.fasta"

batch_size = 32
max_seq_len = 2000
data_gen = data_generator(large_file, max_seq_len, acids)

In [ ]:
for i in range(1000):
    data_gen.gen_data(batch_size)

In [ ]:
batches = 1

# For some reason this keeps hogging memory in each iteration, don't know why
for i in range(batches):
    model.zero_grad()

    data_gen.gen_data(batch_size)
    sequence_batch = data_gen.data#.cuda()
    print(sequence_batch.size())
    output = model(sequence_batch)

    # This line will complain about wrong dimensions, don't know why though
    loss = loss_function(output, data_gen.targets)
    loss.backward()
    optimiser.step()

In [ ]:
print(data_gen.data.shape)
seqs = data_gen.data.view(max_seq_len,batch_size,21)
seqs = seqs.type(torch.float)
print(type(seqs))
seqs = seqs.to(device='cuda')
test = LSTM_model(21,500,1)
model.zero_grad()


test_data = test.forward(seqs)


#output = test.forward(char_seqs)
#output = output.detach().numpy()
#argmax = np.argmax(output[0])
#print(argmax)
#print(output[0][argmax])